In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
from pandas import DataFrame


In [2]:
url= 'https://www.imdb.com/search/title/?num_votes=100000,&sort=user_rating,desc&title_type=tv_series&'
data=requests.get(url)
news_data = []
soup = BeautifulSoup(data.content, 'html.parser')

# Creating database

In [3]:
name=[]
certi=[]#null
rate=[]
run=[]
genre=[]
gross=[]#null
director=[]
summary=[]


movie_Data=soup.find_all('div',attrs = {'class':'lister-item-content'})

for data in movie_Data:
    name.append(data.h3.a.text)
    run.append(data.find('span', attrs = {'class':'runtime'}).text)
    rating=data.find('div', attrs = {'class':'inline-block ratings-imdb-rating'})
    rate.append(rating['data-value'])
    genre.append(data.find('span', attrs = {'class':'genre'}).text.replace('\n', ' '))
    temp=data.find('p',attrs = {'class':''})
    director.append(temp.a.text)    
    temp1=data.find_all('p',{'class':'text-muted'})
    temp1=temp1[-1].text
    summary.append(str(temp1.replace('\n','').strip()))
    
df = pd.DataFrame({'Movie_Name':name,'rating':rate,'runtime':run,'genre':genre,'Director':director,'Summary':summary})
df.head()

df.to_csv('preview.csv', index=False)  

In [5]:
df.head()

,Movie_Name,rating,runtime,genre,Director,Summary
0,Breaking Bad,9.5,49 min,"Crime, Drama, Thriller",Bryan Cranston,A high school chemistry teacher diagnosed with...
1,Cosmos: A Spacetime Odyssey,9.3,557 min,Documentary,Neil deGrasse Tyson,An exploration of our discovery of the laws of...
2,Game of Thrones,9.3,57 min,"Action, Adventure, Drama",Emilia Clarke,Nine noble families fight for control over the...
3,The Wire,9.3,59 min,"Crime, Drama, Thriller",Dominic West,"The Baltimore drug scene, as seen through the ..."
4,Rick and Morty,9.2,23 min,"Animation, Adventure, Comedy",Justin Roiland,An animated series that follows the exploits o...


# Pre-processing
Expanding Contraction,
Remove Special char

In [11]:
import re
from contractions import CONTRACTION_MAP

contractions_re = re.compile('(%s)' % '|'.join(CONTRACTION_MAP.keys()))

def expand_contractions(s, contractions_dict=CONTRACTION_MAP):
    def replace(match):
            return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


text=("A gangster family epic set in 1919 Birmingham, England; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss Tommy Shelby.").lower()
expand_contractions(text)


'a gangster family epic set in 1919 birmingham, england; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss tommy shelby.'

In [12]:

def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

remove_special_characters("A gangster family epic set in 1919 Birmingham, England; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss Tommy Shelby.")

'A gangster family epic set in 1919 Birmingham England centered on a gang who sew razor blades in the peaks of their caps and their fierce boss Tommy Shelby'

Stemming and Lemmatization 

In [13]:
#stemming
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

def simple_stemmer(text):
    words = word_tokenize(text) 
    for w in words: 
        #print(w, " : ", PorterStemmer().stem(w)) 
        text=re.sub(w,PorterStemmer().stem(w),text,count=1)
    return text

simple_stemmer("Eat Eating Playing played play playing Stemming stem stemmed rocks corpora better.")

'.at eat play play play play stem stem stem rock corpora better.'

In [14]:
#Lemmatization (takes part of speech(pos) - default noun)
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
  
print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora")) 
  
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a")) 
print("playing :", lemmatizer.lemmatize("playing", pos ="v")) 
print("playing :", lemmatizer.lemmatize("playing", pos ="a")) 
def simple_lemmatization(text):
    words = word_tokenize(text) 
    for w in words: 
        #print(w, " : ", PorterStemmer().stem(w)) 
        text=re.sub(w,lemmatizer.lemmatize(w),text,count=2)
    return text

simple_lemmatization("Fishing at swimming at the banks of river rocks corpora better")

rocks : rock
corpora : corpus
better : good
playing : play
playing : playing


'Fishing at swimming at the bank of river rock corpus better'

# Stop words

In [15]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

stopword_list = nltk.corpus.stopwords.words('english')
#stopword_list.remove('no')
#stopword_list.remove('not')
#stopword_list.remove('this')
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

remove_stopwords("This is a sample sentence, showing off the stop words filtration.")

'sample sentence , showing stop words filtration .'

# Finding part of Speech (POS)

CC coordinating conjunction

CD cardinal digit

DT determiner

EX existential there (like: “there is” … think of it like “there exists”)

FW foreign word

IN preposition/subordinating conjunction

JJ adjective ‘big’

JJR adjective, comparative ‘bigger’

JJS adjective, superlative ‘biggest’

LS list marker 1)

MD modal could, will

NN noun, singular ‘desk’

NNS noun plural ‘desks’

NNP proper noun, singular ‘Harrison’

NNPS proper noun, plural ‘Americans’

PDT predeterminer ‘all the kids’

POS possessive ending parent‘s

PRP personal pronoun I, he, she

PRP$ possessive pronoun my, his, hers

RB adverb very, silently,

RBR adverb, comparative better

RBS adverb, superlative best

RP particle give up

TO to go ‘to‘ the store.

UH interjection errrrrrrrm

VB verb, base form take

VBD verb, past tense took

VBG verb, gerund/present participle taking

VBN verb, past participle taken

VBP verb, sing. present, non-3d take

VBZ verb, 3rd person sing. present takes

WDT wh-determiner which

WP wh-pronoun who, what

WP$ possessive wh-pronoun whose

WRB wh-abverb where, when

The heavens are above. (Adverb)

The moral code of conduct is above the civil code of conduct. (Preposition)

Read the sentence given above. (Adjective)

Our blessings come from above. (Noun)

In [50]:
#blob
from textblob import TextBlob 
  
text = ("The heavens are above. The moral code of conduct is above the civil code of conduct. Read the sentence given above. Our blessings come from above.") 
  
# create a textblob object 
blob_object = TextBlob(text) 
  
# Part-of-speech tags can be accessed  
# through the tags property of blob object.' 
  
# print word with pos tag. 
print(blob_object.tags) 

[('The', 'DT'), ('heavens', 'NNS'), ('are', 'VBP'), ('above', 'IN'), ('The', 'DT'), ('moral', 'JJ'), ('code', 'NN'), ('of', 'IN'), ('conduct', 'NN'), ('is', 'VBZ'), ('above', 'IN'), ('the', 'DT'), ('civil', 'JJ'), ('code', 'NN'), ('of', 'IN'), ('conduct', 'NN'), ('Read', 'VB'), ('the', 'DT'), ('sentence', 'NN'), ('given', 'VBN'), ('above', 'IN'), ('Our', 'PRP$'), ('blessings', 'NNS'), ('come', 'VBP'), ('from', 'IN'), ('above', 'IN')]
<class 'list'>


In [68]:
df_blob=pd.DataFrame(blob_object.tags,columns=['origional','blob'])
df_blob

,origional,blob
0,The,DT
1,heavens,NNS
2,are,VBP
3,above,IN
4,The,DT
5,moral,JJ
6,code,NN
7,of,IN
8,conduct,NN
9,is,VBZ


In [64]:
#pos tagger nltk
tokens=nltk.word_tokenize("The heavens are above. The moral code of conduct is above the civil code of conduct. Read the sentence given above. Our blessings come from above.")
print(nltk.pos_tag(tokens))

df_spacy_nltk=pd.DataFrame(nltk.pos_tag(tokens),columns=['Origional',"nltk"])

[('The', 'DT'), ('heavens', 'NNS'), ('are', 'VBP'), ('above', 'IN'), ('.', '.'), ('The', 'DT'), ('moral', 'JJ'), ('code', 'NN'), ('of', 'IN'), ('conduct', 'NN'), ('is', 'VBZ'), ('above', 'IN'), ('the', 'DT'), ('civil', 'JJ'), ('code', 'NN'), ('of', 'IN'), ('conduct', 'NN'), ('.', '.'), ('Read', 'VB'), ('the', 'DT'), ('sentence', 'NN'), ('given', 'VBN'), ('above', 'IN'), ('.', '.'), ('Our', 'PRP$'), ('blessings', 'NNS'), ('come', 'VBP'), ('from', 'IN'), ('above', 'IN'), ('.', '.')]


In [65]:
#spacy
import spacy 
spacy_df=[]
# Load English tokenizer, tagger,  
# parser, NER and word vectors 
nlp = spacy.load('en_core_web_sm') 
  
# Process whole documents 
text = ("The heavens are above. The moral code of conduct is above the civil code of conduct. Read the sentence given above. Our blessings come from above.") 
doc = nlp(text) 
  
# Token and Tag 
for token in doc:
    #print(token, token.pos_)
    spacy_df.append(token.pos_)
    
#print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"]) 

df_spacy_nltk['spacy']=spacy_df
df_spacy_nltk

,Origional,nltk,spacy
0,The,DT,DET
1,heavens,NNS,NOUN
2,are,VBP,AUX
3,above,IN,ADJ
4,.,.,PUNCT
5,The,DT,DET
6,moral,JJ,ADJ
7,code,NN,NOUN
8,of,IN,ADP
9,conduct,NN,NOUN


# Name Entity Recognition

spaCy supports the following entity types:

PERSON

NORP (nationalities, religious and political groups)

FAC (buildings, airports etc.)

ORG (organizations)

GPE (countries, cities etc.)

LOC (mountain ranges, water bodies etc.)

PRODUCT (products)

EVENT (event names)

WORK_OF_ART (books, song titles)

LAW (legal document titles)

LANGUAGE (named languages)

DATE, TIME, PERCENT, MONEY, QUANTITY, ORDINAL and CARDINAL.



In [13]:
import spacy 

nlp = spacy.load('en_core_web_sm') 

sentence = "A gangster family epic set in 1919 Birmingham, England; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss Tommy Shelby."

doc = nlp(sentence) 

for ent in doc.ents: 
    #print(ent.text, ent.start_char, ent.end_char, ent.label_) 
    print(ent.text, ent.label_) 


1919 DATE
Birmingham GPE
England GPE
Tommy Shelby PERSON


# TF-IDF
TF = (Number of times term t appears in a document)/(Number of terms in the document)

IDF = log(Number of documents/Number of documents in which term t has appeared)

TF-IDF=TF*IDF

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
documentA = 'This is about Messi'
documentB = 'This is about TFIDF'
bagOfWordsA = documentA.split(' ')
bagOfWordsB = documentB.split(' ')
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB))

numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1
    
numOfWordsB = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsB:
    numOfWordsB[word] += 1

In [33]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [34]:
tfA = computeTF(numOfWordsA, bagOfWordsA)
tfB = computeTF(numOfWordsB, bagOfWordsB)

print(tfA)
print(tfB)

{'about': 0.25, 'This': 0.25, 'TFIDF': 0.0, 'Messi': 0.25, 'is': 0.25}
{'about': 0.25, 'This': 0.25, 'TFIDF': 0.25, 'Messi': 0.0, 'is': 0.25}


In [35]:
#IDF
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [36]:
idfs = computeIDF([numOfWordsA, numOfWordsB])
print(idfs)

{'about': 0.0, 'This': 0.0, 'TFIDF': 0.6931471805599453, 'Messi': 0.6931471805599453, 'is': 0.0}


In [37]:
#TF-IDF
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
df = pd.DataFrame([tfidfA, tfidfB])

In [38]:
df

,about,This,TFIDF,Messi,is
0,0.0,0.0,0.000000,0.173287,0.0
1,0.0,0.0,0.173287,0.000000,0.0


# TF-IDF using sklearn

In [39]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [40]:
df

,about,is,messi,tfidf,this
0,0.448321,0.448321,0.630099,0.000000,0.448321
1,0.448321,0.448321,0.000000,0.630099,0.448321


# Sentiment analysis using TextBlob
Polarity range [-1,1]

In [13]:
from textblob import TextBlob

testimonial = TextBlob("This apple is not good")
print(testimonial.sentiment)
testimonial = TextBlob("This apple is good")
testimonial.sentiment

Sentiment(polarity=-0.35, subjectivity=0.6000000000000001)


Sentiment(polarity=0.7, subjectivity=0.6000000000000001)